<a href="https://colab.research.google.com/github/dajopr/lectures/blob/main/image_processing/lecture_05_feature_matching_morphology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise Sheet: Local Feature Detection, Description, Matching & Homography

Prerequisites: Basic Python, NumPy, OpenCV (image loading, display, color conversion, drawing functions), Matplotlib. Familiarity with basic image processing concepts.

Goal: To understand, implement, and evaluate methods for detecting and describing local image features (e.g., SIFT, ORB), matching these features between images robustly, and computing homography transformations for applications like image alignment and perspective analysis.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


def show_images(
    images, titles, rows, cols, figsize=(15, 10), main_title="Image Comparison"
):
    """Helper function to display multiple images using Matplotlib"""
    if not images:
        print("No images to display.")
        return
    if len(images) != len(titles):
        print("Warning: Number of images and titles do not match.")
        min_count = min(len(images), len(titles))
        images = images[:min_count]
        titles = titles[:min_count]

    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    fig.suptitle(main_title, fontsize=16)

    if rows * cols == 1:  # Handle single subplot case
        axes = np.array([axes])  # Make it iterable like multiple subplots
    axes = axes.ravel()  # Flatten the axes array for easy iteration

    plot_index = 0
    for i, img in enumerate(images):
        if plot_index < len(axes):
            current_ax = axes[plot_index]
            if img is not None:
                # Convert BGR (OpenCV default) to RGB for Matplotlib display
                if len(img.shape) == 2 or (
                    len(img.shape) == 3 and img.shape[2] == 1
                ):  # Grayscale
                    current_ax.imshow(img, cmap="gray")
                    current_ax.set_title(titles[i])
                elif len(img.shape) == 3 and img.shape[2] == 3:  # Color (BGR)
                    current_ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
                    current_ax.set_title(titles[i])
                elif (
                    len(img.shape) == 3 and img.shape[2] == 4
                ):  # Color with Alpha (BGRA)
                    current_ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGRA2RGBA))
                    current_ax.set_title(titles[i])
                else:
                    current_ax.text(
                        0.5,
                        0.5,
                        "Unsupported Format",
                        horizontalalignment="center",
                        verticalalignment="center",
                        transform=current_ax.transAxes,
                    )
                    current_ax.set_title(f"{titles[i]} (Unsupported Format)")
            else:
                current_ax.text(
                    0.5,
                    0.5,
                    "Image is None",
                    horizontalalignment="center",
                    verticalalignment="center",
                    transform=current_ax.transAxes,
                )
                current_ax.set_title(f"{titles[i]} (None)")
            current_ax.axis("off")
            plot_index += 1
        else:
            print(
                f"Warning: More images provided than subplot slots ({rows * cols}). Skipping '{titles[i]}'."
            )

    # Turn off any remaining unused subplots
    for j in range(plot_index, len(axes)):
        axes[j].axis("off")

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust layout to make room for suptitle
    plt.show()


# Exercise 1: Introduction to SIFT (Scale-Invariant Feature Transform)

**Objective:** To understand and apply the SIFT (Scale-Invariant Feature Transform) algorithm for keypoint detection and descriptor computation using OpenCV. This exercise will cover initializing the SIFT detector, finding keypoints, computing their descriptors, and visualizing the results.

**Instructions:**

1.  **Load and Prepare Image:**
    * Choose one of your test images.
    * In your Python script, load it as a color image (e.g., using `cv2.imread()`) and assign it to a variable named `image_color`. You can use `sunflowers.jpg` or any image of your choosing.
    * Create a grayscale version of this image (e.g., using `cv2.cvtColor()`) and assign it to a variable named `image_gray`.
    * It's good practice to check if the image was loaded successfully.

2.  **Initialize SIFT Detector:**
    * Create a SIFT detector object using the appropriate OpenCV function (e.g., `cv2.SIFT_create()`).
    * Assign this detector object to a variable named `sift`.

3.  **Detect Keypoints:**
    * Using the `sift` object's detection method (e.g., `sift.detect()`) and the `image_gray`, detect keypoints.
    * Assign the resulting list of keypoints to a variable named `keypoints_sift`.
    * You can print the number of detected keypoints using `len(keypoints_sift)`.

4.  **Examine Keypoint and Descriptor Details:**
    * Print the total number of keypoints found by SIFT.
    * Print the shape of a `keypoint_sift`.
    * *(Optional)* To understand what a keypoint object contains, access an individual keypoint from the `keypoints_sift` list (e.g., `keypoints_sift[0]`) and inspect its attributes like `pt` (coordinates), `size`, `angle`, `response`, and `octave`.

5.  **Visualize Keypoints:**
    * Draw the detected `keypoints_sift` on the original `image_color`. OpenCV provides a function for this (e.g., `cv2.drawKeypoints()`).
    * To see the scale and orientation of keypoints, use a flag like `cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS` when drawing.
    * Display the resulting image (with keypoints drawn on it) using a library like Matplotlib (e.g., using `plt.imshow()`). Remember to handle color channel conversion if necessary (OpenCV uses BGR by default, Matplotlib uses RGB).

In [ ]:
def print_sift_keypoint_details(keypoint):
    """
    Prints the relevant fields of a SIFT keypoint descriptor.

    Args:
        keypoint (cv2.KeyPoint): A single SIFT keypoint object.
    """
    if not isinstance(keypoint, cv2.KeyPoint):
        print("Invalid input: Expected a cv2.KeyPoint object.")
        return

    print(f"Keypoint Details:")
    print(f"  - Location (x, y): {keypoint.pt}")
    print(f"  - Size: {keypoint.size}")
    print(f"  - Angle: {keypoint.angle}")
    print(f"  - Response: {keypoint.response}")
    print(f"  - Octave: {keypoint.octave}")
    print(f"  - Class ID: {keypoint.class_id}")

In [ ]:
image_path = "IMAGE.png"  # Change this to selected image

image_color = cv2.imread(image_path, cv2.IMREAD_COLOR_RGB)
image_gray = cv2.cvtColor(image_color, cv2.COLOR_RGB2GRAY)

# 2. Initialize SIFT detector
sift = None  # Change this
sift = cv2.SIFT_create()


# 3. Detect keypoints
keypoints_sift = None  # Change this

# 4. Print number of keypoints and shape of a keypoint object
num_keypoints = None  # Change this
first_keypoint = None  # Change this

print(f"Number of keypoints: {num_keypoints}")
print_sift_keypoint_details(first_keypoint)

# Exercise 2: Comparing Keypoint Detectors (SIFT, ORB, AKAZE)

**Objective:** To apply and compare SIFT, ORB, and AKAZE feature detection algorithms on various images, observing their characteristics, the number of keypoints detected, their distribution, and their suitability for different image types.

**Prerequisites:**

* Python environment with OpenCV and Matplotlib.
* **OpenCV Installation:** Ensure you have the necessary OpenCV packages. SIFT is included in the `opencv-contrib-python` package. If you don't have it, you might need to install or upgrade:
    ```bash
    pip install opencv-python opencv-contrib-python matplotlib
    ```
* **Test Images:** Download the test images (or select your own). Consider images with:
    1.  Good, varied textures and distinct features (e.g., `landscape_with_buildings.jpg` or `sunflowers.jpg`).
    2.  Significant scale changes (e.g., `taj_mahal_far.jpg`, `taj_mahal_close.jpg`).
    3.  Rotation (e.g., `book_frontal.png`, `book_rotated.png`).
    4.  Perspective distortion (e.g., `sticker_frontal.jpg`, `sticker_angle.jpg`).
    5.  Blurred (e.g.`sticker_blurred.jpg` )

**Instructions:**

1.  **Load and Prepare Image:**
    * Choose one of your test images.
    * Load it as a color image and assign it to a variable named `image_color`.
    * Create a grayscale version of this image and assign it to a variable named `image_gray`.

2.  **Initialize Detectors:**
    * Create a default SIFT detector object and assign it to a variable named `sift`.
    * Create a default ORB detector object and assign it to a variable named `orb`.
        * *(Optional: You can experiment with `nfeatures` for ORB, e.g., `cv2.ORB_create(nfeatures=1000)` after trying the default).*
    * Create a default AKAZE detector object and assign it to a variable named `akaze`.

3.  **Detect Keypoints (Features):**
    * Using the `sift` object and `image_gray`, detect keypoints and assign the resulting list of keypoints to a variable named `keypoints_sift`.
    * Using the `orb` object and `image_gray`, detect keypoints and assign them to a variable named `keypoints_orb`.
    * Using the `akaze` object and `image_gray`, detect keypoints and assign them to a variable named `keypoints_akaze`.

4.  **Examine Keypoint Counts:**
    * Print the number of keypoints found by each detector. For example:
        `print(f"SIFT detected {len(keypoints_sift)} keypoints.")`
        `print(f"ORB detected {len(keypoints_orb)} keypoints.")`
        `print(f"AKAZE detected {len(keypoints_akaze)} keypoints.")`

5.  **Visualize Keypoints:**
    * The provided code (in the Jupyter Notebook, to be supplied by your instructor) will take `image_color`, `keypoints_sift`, `keypoints_orb`, and `keypoints_akaze` to draw these keypoints on separate copies of the image and display them in a single plot for comparison. Ensure your variables are named correctly for the provided display code to work.

6.  **Experiment:**
    * After observing the results on your first selected image, run the same detection steps (1-4) on a different image that has distinct characteristics (e.g., more/less texture, significant rotation or scale difference compared to an imaginary "original"). This will help inform your answers to the discussion questions.

7.  **Answer the Discussion Questions.**


In [ ]:
image_path = "images/IMAGE.jpg"  # <<< CHANGE THIS TO YOUR IMAGE PATH

image_color = cv2.imread(image_path, cv2.IMREAD_COLOR)
image_color = cv2.resize(image_color, None, fx=0.5, fy=0.5)
if image_color is None:
    raise FileNotFoundError(f"Image not found or unable to read: {image_path}")
image_gray = cv2.cvtColor(image_color, cv2.COLOR_BGR2GRAY)
# --- Student Code Starts Here ---

# Create instances of SIFT, ORB (nfeatures=500) and AKAZE detectors
sift = None
orb = None
akaze = None


# Detect keypoints using the detectors
keypoints_sift = []
keypoints_orb = []
keypoints_akaze = []


# --- Student Code Ends Here ---
img_display_sift = image_color.copy()
img_display_orb = image_color.copy()
img_display_akaze = image_color.copy()

# Draw SIFT keypoints
if sift and keypoints_sift:  # Check if sift object and keypoints exist
    cv2.drawKeypoints(
        image_color,
        keypoints_sift,
        img_display_sift,
        flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS,
    )
elif not sift:  # SIFT object itself failed to create
    cv2.putText(
        img_display_sift,
        "SIFT Not Available",
        (50, int(img_display_sift.shape[0] / 2)),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 0, 255),
        2,
    )
else:  # SIFT created but no keypoints or other error
    cv2.putText(
        img_display_sift,
        "No SIFT Keypoints",
        (50, int(img_display_sift.shape[0] / 2)),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 0, 255),
        2,
    )


# Draw ORB keypoints
if orb and keypoints_orb:
    cv2.drawKeypoints(
        image_color,
        keypoints_orb,
        img_display_orb,
        color=(0, 255, 0),
        flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS,
    )
else:
    cv2.putText(
        img_display_orb,
        "No ORB Keypoints / Error",
        (50, int(img_display_orb.shape[0] / 2)),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 255, 0),
        2,
    )


# Draw AKAZE keypoints
if akaze and keypoints_akaze:
    cv2.drawKeypoints(
        image_color,
        keypoints_akaze,
        img_display_akaze,
        color=(255, 0, 0),
        flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS,
    )
else:
    cv2.putText(
        img_display_akaze,
        "No AKAZE Keypoints / Error",
        (50, int(img_display_akaze.shape[0] / 2)),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (255, 0, 0),
        2,
    )


# Display using the helper function
images_to_show = [image_color, img_display_sift, img_display_orb, img_display_akaze]
titles_to_show = [
    "Original Color",
    f"SIFT Keypoints ({len(keypoints_sift) if keypoints_sift else '0/Error'})",
    f"ORB Keypoints ({len(keypoints_orb) if keypoints_orb else '0/Error'})",
    f"AKAZE Keypoints ({len(keypoints_akaze) if keypoints_akaze else '0/Error'})",
]

show_images(
    images_to_show,
    titles_to_show,
    2,
    2,
    figsize=(18, 12),
    main_title="Feature Detector Comparison",
)

**Discussion Questions:**

1. **Quantity:** Which detector produced the most keypoints on your primary test image? Which produced the fewest? If you experimented with `nfeatures` for ORB, how did it affect the count?
2. **Distribution:** Based on the visualization (from the provided display code):
    * Observe the locations of the keypoints for each detector. Are they clustered in certain areas or spread out?
    * Do they tend to be on corners, edges, or in textured regions? Describe any differences you see between SIFT, ORB, and AKAZE in terms of *where* they find features on your test image.
3. **Keypoint Characteristics:** The visualization (if using `cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS` in the provided display code) shows the size and orientation of keypoints.
    * Do all detectors provide scale information (indicated by circle size)?
    * Do all detectors provide orientation information (indicated by the line radiating from the center)?
4. **Computational Speed (Qualitative):** While not explicitly measured, did you notice any significant differences in how long each detector took to run?
5. **Robustness (Conceptual, based on your image or recalling detector properties):**
    * Consider an image with significant **scale changes** (e.g., an object seen from far away and then close up). Which detector (SIFT, ORB, AKAZE) would you expect to be most robust in consistently finding corresponding features? Why?
    * Which detector is particularly known for its speed and efficiency in detecting **corners**?
    * AKAZE uses a non-linear scale space. How might this influence the types of features it detects or its robustness compared to SIFT's Gaussian scale space?
6. **Parameters:**
    * Besides `nfeatures`, what other important parameters can you configure for `cv2.ORB_create()`? (Consult the OpenCV documentation).
    * Do `cv2.SIFT_create()` or `cv2.AKAZE_create()` offer parameters to control feature detection (e.g., sensitivity, number of octaves/layers)? Briefly check their documentation.
7. **Use Cases:** Based on your observations and understanding of their properties:
    * Suggest a scenario or type of application where ORB might be the preferred choice.
    * Suggest a scenario where SIFT or AKAZE might be more suitable, even if they are computationally more intensive.

# Exercise 2: Feature Matching Between Two Images

**Objective:** To detect features in two images of the same scene (e.g., a building) and match them using OpenCV. This exercise will primarily use the SIFT detector, the Brute-Force (BF) matcher, apply Lowe's Ratio Test for robust matching, and visualize the results. Comparisons with AKAZE or ORB can be explored as optional extensions.

**Instructions:**
1.  **Load and Prepare Images:**
    * Load your two images (e.g., `sticker_frontal.jpg` and `sticker_rotated.jpg`) into color variables (e.g., `image1_color`, `image2_color`).
    * Convert both color images to grayscale (e.g., `image1_gray`, `image2_gray`). Grayscale is typically used for feature detection.
    * Check if images loaded correctly.

2.  **Initialize Detector (SIFT):**
    * Create an SIFT detector object (e.g., using `cv2.SIFT_create()`). You can optionally specify parameters like `nfeatures` (e.g., `cv2.SIFT_create(nfeatures=1000)`).
    * Assign it to a variable like `sift`.

3.  **Detect Keypoints and Compute Descriptors for Both Images:**
    * For the first grayscale image (`image1_gray`):
        * Use the `sift` object's method to detect keypoints and compute their descriptors simultaneously (e.g., `sift.detectAndCompute(image1_gray, None)`).
        * Store the results in variables like `kp1` (keypoints for image 1) and `des1` (descriptors for image 1).
    * Repeat the process for the second grayscale image (`image2_gray`):
        * Store its keypoints and descriptors in variables like `kp2` and `des2`.
    * Print the number of keypoints detected in each image (e.g., using `len(kp1)`). Check if `des1` and `des2` are not `None`.

4.  **Initialize Brute-Force Matcher:**
    * Create a Brute-Force matcher object (e.g., `cv2.BFMatcher()`).
    * Set `crossCheck=False`. We will use k-Nearest Neighbor matching and then apply Lowe's ratio test, which requires finding more than one match per keypoint. If `crossCheck=True`, the matcher only returns mutually best matches.

5.  **Match Descriptors using k-NN:**
    * Use the Brute-Force matcher's k-Nearest Neighbors matching method (e.g., `bf.knnMatch()`) to find the `k=2` best matches from `des2` for each descriptor in `des1`.
    * Store these matches in a variable, for example, `matches_knn`. Each element in `matches_knn` will be a list containing two `DMatch` objects (the two best matches), provided at least two matches were found.

6.  **Apply Lowe's Ratio Test to Filter Good Matches:**
    * Create an empty list called `good_matches`.
    * Iterate through each pair of matches `(m, n)` in your `matches_knn` list. `m` is the best match, `n` is the second-best match.
    * For a match `m` to be considered "good", its distance should be significantly smaller than the distance of the second-best match `n`. The condition is: `m.distance < ratio * n.distance`.
    * A common value for `ratio` is between 0.7 and 0.8. Start with `ratio = 0.75`.
    * If a match `m` satisfies this condition, append `m` (the first, better match object) to your `good_matches` list.
    * Print the number of raw matches found by `knnMatch` (e.g., `len(matches_knn)`) and the number of matches remaining after applying the ratio test (e.g., `len(good_matches)`).

7.  **Visualize Good Matches:**
    * Use OpenCV's function to draw the matches (e.g., `cv2.drawMatches()`).
    * This function takes the two original color images (`image1_color`, `image2_color`), their respective keypoints (`kp1`, `kp2`), the list of good matches (`good_matches`), and an output image object (can be `None`).
    * You can also use flags, for instance, `cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS` can make the visualization cleaner if there are many matches.
    * Display the resulting image (which shows lines connecting matched keypoints across the two input images) using Matplotlib. Remember BGR to RGB conversion for display.

**Optional Extensions:**
* **Compare with AKAZE:**
    1.  Replace the SIFT detector with an AKAZE detector (e.g., `akaze = cv2.AKAZE_create()`).
    2.  Detect keypoints and compute descriptors using AKAZE for both images.
    3.  The default AKAZE descriptors are binary (similar to ORB), so you can typically use `cv2.NORM_HAMMING` for matching. (Note: AKAZE can also produce float descriptors if configured differently).
    4.  Apply the same `knnMatch` and Lowe's Ratio Test.
    5.  Compare the number and visual quality of matches with those obtained using SIFT.

* **Compare with SIFT (if `opencv-contrib-python` is installed):**
    1.  Replace the SIFT detector with a ORB detector (e.g., `orb = cv2.ORB_create()`).
    2.  Detect keypoints and compute ORB descriptors.
    3.  The ORB descriptors are binary (similar to default in AKAZE), so you can typically use `cv2.NORM_HAMMING` for matching. (Note: AKAZE can also produce float descriptors if configured differently).
    4.  Apply `knnMatch` and Lowe's Ratio Test as before.
    5.  Compare with SIFT and AKAZE. Consider differences in the number of keypoints, the nature of the keypoints, computational time and match quality.

In [ ]:
# --- 1. Load and Prepare Images ---
image1_color = cv2.imread("images/sticker_rotated.jpg")
image2_color = cv2.imread("images/sticker_frontal.jpg")

if image1_color is None or image2_color is None:
    print("Error: Could not load one or both images. Check the paths.")
    exit()

image1_gray = cv2.cvtColor(image1_color, cv2.COLOR_BGR2GRAY)
image2_gray = cv2.cvtColor(image2_color, cv2.COLOR_BGR2GRAY)

# --- 2. Initialize Detector (SIFT) ---
# You can experiment with nfeatures; default is 500
sift = None  # Change this

# --- 3. Detect Keypoints and Compute Descriptors ---
kp1, des1 = (None, None)  # Change this
kp2, des2 = (None, None)  # Change this


if des1 is None or des2 is None:
    print("Error: Could not compute descriptors for one or both images.")
    if len(kp1) == 0:
        print("No keypoints found in image 1.")
    if len(kp2) == 0:
        print("No keypoints found in image 2.")
    exit()

print(f"Keypoints detected in image 1: {len(kp1)}")
print(f"Keypoints detected in image 2: {len(kp2)}")

print(f"Descriptors shape for image 1: {des1.shape}")
print(f"Descriptors shape for image 2: {des2.shape}")


# --- 4. Initialize Brute-Force Matcher ---
# SIFT uses real valued descriptors, so NORM_L2 is used.
# crossCheck=False because we are using knnMatch for Lowe's ratio test.

bf = None  # Change this

# --- 5. Match Descriptors using k-NN ---
# Find k=2 best matches for each descriptor in des1 from des2.

matches_knn = None  # Change this

print(f"Number of raw matches (k=2) from knnMatch: {len(matches_knn)}")

# --- 6. Apply Lowe's Ratio Test ---
good_matches = []
ratio_thresh = 0.8  # Lowe's ratio threshold

# Ensure that for each descriptor, at least two matches were found
# (i.e., the list associated with each descriptor has length 2)

good_matches = []  # Change this

print(f"Number of good matches after Lowe's ratio test: {len(good_matches)}")

# --- 7. Visualize Good Matches ---
# cv2.drawMatches needs a list of DMatch objects.
# The 'good_matches' list already contains these.

img_matches_orb = None  # Change this

# Display the matches
plt.figure(figsize=(16, 8))
plt.imshow(cv2.cvtColor(img_matches_orb, cv2.COLOR_BGR2RGB))
plt.title("Good Matches (SIFT + Lowe's Ratio Test)")
plt.axis("off")
plt.show()


**Discussion Questions**

1.  How many keypoints were detected in `image1` and `image2` respectively?
2.  How many raw matches were found by `knnMatch` before applying the ratio test?
3.  How many "good" matches remained after applying Lowe's Ratio Test with your chosen `ratio`?
4.  **Experiment with the `ratio` for Lowe's Test:**
    * Change the `ratio` value (e.g., to 0.6, 0.7, 0.8, 0.9). How does this affect the number of good matches?
    * Visually inspect the quality of matches for different ratios. What do you observe about stricter (lower ratio) vs. more lenient (higher ratio) filtering?
5.  Why is Lowe's Ratio Test used? What problem does it help to solve in feature matching?
6.  What does the `cv2.NORM_HAMMING` distance measure? Why is it suitable for ORB

# Exercise 3: Image Alignment with Homography

**Objective:** To compute a homography matrix from matched feature points between two images and use this matrix to warp one image to align its perspective with the other. This exercise will cover the use of `cv2.findHomography` with RANSAC and `cv2.warpPerspective`.

**Prerequisites:**

* Python environment with OpenCV, NumPy, and Matplotlib installed.
* **Test Images & Matches:** You will need two images of the same planar scene or an object taken with only camera rotation (e.g., `building_view1.jpg`, `building_view2.jpg`). You'll also need a set of good feature matches between them. This exercise ideally follows "Exercise 2: Feature Matching." You should have:
    * `image1_color`, `image2_color` (the original color images)
    * `kp1`, `kp2` (keypoints for image1 and image2 respectively)
    * `good_matches` (a list of `DMatch` objects representing good matches, e.g., after Lowe's Ratio Test)
* Ensure you have at least 4 good matches, as this is the minimum required to compute a homography.

**Instructions:**

1.  **Prepare Matched Points (Continuing from Exercise 2):**
    * If you haven't already, run your feature matching code (e.g., using SIFT with Lowe's Ratio Test from Exercise 2) to obtain `image1_color`, `image2_color`, `kp1`, `kp2`, and the list `good_matches`.
    * Verify that `len(good_matches)` is at least 4. If not, you might need to adjust matching parameters (e.g., the ratio for Lowe's test, number of features) or use images with more overlap/features.

2.  **Extract Coordinates of Matched Keypoints:**
    * Create two lists (or NumPy arrays) of corresponding points:
        * `src_pts`: Coordinates of keypoints from `good_matches` in the first image (`image1_color`). These are the points that will be transformed.
        * `dst_pts`: Coordinates of keypoints from `good_matches` in the second image (`image2_color`). These are the target points in the reference perspective.
    * For each `DMatch` object `m` in `good_matches`:
        * The point in the first image is `kp1[m.queryIdx].pt`.
        * The point in the second image is `kp2[m.trainIdx].pt`.
    * Convert these lists of points to NumPy arrays of shape `(N, 1, 2)` and type `float32`.

3.  **Compute Homography Matrix using RANSAC:**
    * Use `cv2.findHomography()` to calculate the homography matrix `H`.
        * `H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, reprojThresh)`
        * `src_pts`: Source points (from `image1_color`).
        * `dst_pts`: Destination points (from `image2_color`).
        * `cv2.RANSAC`: Specifies the RANSAC algorithm to make the estimation robust to outliers.
        * `reprojThresh`: Maximum allowed reprojection error for a point pair to be considered an inlier (e.g., `5.0`). Experiment with this value.
    * The function returns the 3x3 homography matrix `H` and a `mask` array. The mask indicates which of the `good_matches` were considered inliers by RANSAC.
    * Print the computed homography matrix `H`.
    * Count and print the number of inliers using the `mask`.

4.  **Warp the Source Image:**
    * Use `cv2.warpPerspective()` to apply the homography `H` to `image1_color` (the source image). This will transform `image1_color` to align with the perspective of `image2_color` (the destination/reference image).
        * `warped_image = cv2.warpPerspective(image1_color, H, (width_ref, height_ref))`
        * `image1_color`: The image to be warped.
        * `H`: The homography matrix.
        * `(width_ref, height_ref)`: The dimensions (width, height) of the destination image (`image2_color`). You can get these from `image2_color.shape[1]` and `image2_color.shape[0]`.
    * Store the result in a variable, e.g., `image1_warped_to_image2`.

5.  **Visualize the Alignment:**
    * **Option A: Side-by-Side:** Display `image2_color` (reference) and `image1_warped_to_image2` side-by-side using Matplotlib for comparison.
    * **Option B: Overlay (Recommended for detailed check):**
        * Create a blended image by overlaying the semi-transparent `image1_warped_to_image2` onto `image2_color`.
        * `blended_image = cv2.addWeighted(image2_color, 0.5, image1_warped_to_image2, 0.5, 0)`
        * Display this `blended_image` using Matplotlib.

**Discussion Questions:**

1.  How many `good_matches` were initially provided to `cv2.findHomography`?
2.  After running RANSAC, how many of these matches were considered inliers (check the `mask` or sum its values)? What does this tell you about the initial set of "good" matches?
3.  Why is RANSAC crucial when computing a homography from feature matches obtained in real-world images? What kind of errors does it help mitigate?
4.  Examine the `image1_warped_to_image2` and the `blended_image`. How accurate is the alignment?
    * Are there any regions where the alignment is poor? What could be the reasons (e.g., non-planar parts of the scene, very few features in some areas, parallax due to camera translation and 3D structure)?
5.  Under what two primary conditions is a homography the mathematically correct model to describe the geometric transformation between two images?
6.  What happens if you try to compute and apply a homography between two images of a non-planar (3D) scene where the camera has undergone significant translation (not just rotation)? Would you expect a perfect global alignment? Why or why not?

In [ ]:
if image1_color is None or image2_color is None:
    print("Error: Could not load one or both images. Check the paths.")
    exit()

# --- 1. Prepare Matched Points (Check minimum matches) ---
MIN_MATCH_COUNT = 4
if len(good_matches) >= MIN_MATCH_COUNT:
    print(f"Sufficient matches found: {len(good_matches)}/{MIN_MATCH_COUNT}")
else:
    print(f"Not enough matches are found - {len(good_matches)}/{MIN_MATCH_COUNT}")
    exit()

# --- 2. Extract Coordinates of Matched Keypoints ---
# Get the coordinates of good matches in both images
# .queryIdx is for the first image (kp1, des1)
# .trainIdx is for the second image (kp2, des2)
src_pts = None  # Change this
dst_pts = None  # Change this

# --- 3. Compute Homography Matrix using RANSAC ---
# reprojThresh: Maximum reprojection error (pixels).
# A common value is between 1.0 to 5.0.
reproj_thresh = 5
H, mask = (None, None)  # Change this

if H is None:
    print("Error: Homography could not be computed. Check your matches or images.")
    exit()

print("\nComputed Homography Matrix H:")
print(H)

# The mask indicates inliers. Count them.
# mask is an array of arrays, e.g. [[1], [0], [1]...]. Summing gives count of inliers.

num_inliers = None  # Change this
print(
    f"Number of inliers identified by RANSAC: {num_inliers} out of {len(good_matches)} original matches"
)

# --- 4. Warp the Source Image ---
# We want to warp image1 to align with image2's perspective.
# The size of the output image should be the size of image2.
height_ref, width_ref = image2_color.shape[:2]
image1_warped_to_image2 = None  # Change this

# --- 5. Visualize the Alignment ---

# Option A: Side-by-Side
fig, axs = plt.subplots(1, 2, figsize=(15, 7))
axs[0].imshow(cv2.cvtColor(image2_color, cv2.COLOR_BGR2RGB))
axs[0].set_title("Reference Image (Image 2)")
axs[0].axis("off")

axs[1].imshow(cv2.cvtColor(image1_warped_to_image2, cv2.COLOR_BGR2RGB))
axs[1].set_title("Image 1 Warped to Image 2 Perspective")
axs[1].axis("off")
plt.suptitle("Homography Alignment: Side-by-Side")
plt.show()

# Option B: Overlay
alpha = 0.6  # Transparency of the warped image
beta = 1.0 - alpha  # Transparency of the reference image
blended_image = cv2.addWeighted(image2_color, alpha, image1_warped_to_image2, beta, 0.0)

plt.figure(figsize=(10, 7))
plt.imshow(cv2.cvtColor(blended_image, cv2.COLOR_BGR2RGB))
plt.title("Blended Overlay: Reference Image and Warped Image")
plt.axis("off")
plt.show()

# (Optional) Draw inlier matches
# Create a new image drawing only the inlier matches
# The mask is used by drawMatches to draw only inlier points.
# Note: mask elements are 0 or 1. Need to convert to list of lists for drawMatches.
matches_mask_for_drawing = mask.ravel().tolist()

img_inlier_matches = cv2.drawMatches(
    image1_color,
    kp1,
    image2_color,
    kp2,
    [m for i, m in enumerate(good_matches) if mask[i][0] == 1],
    None,
    # matchColor=(0, 255, 0), # Green for inliers
    # singlePointColor=None,
    # matchesMask=matches_mask_for_drawing, # This would require good_matches itself
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS,
)

plt.figure(figsize=(16, 8))
plt.imshow(cv2.cvtColor(img_inlier_matches, cv2.COLOR_BGR2RGB))
plt.title("Inlier Matches (RANSAC)")
plt.axis("off")
plt.show()

**Discussion Questions:**

1.  How many `good_matches` were initially provided to `cv2.findHomography`?
2.  After running RANSAC, how many of these matches were considered inliers (check the `mask` or sum its values)? What does this tell you about the initial set of "good" matches?
3.  Why is RANSAC crucial when computing a homography from feature matches obtained in real-world images? What kind of errors does it help mitigate?
4.  Examine the `image1_warped_to_image2` and the `blended_image`. How accurate is the alignment?
    * Are there any regions where the alignment is poor? What could be the reasons (e.g., non-planar parts of the scene, very few features in some areas, parallax due to camera translation and 3D structure)?
5.  Under what two primary conditions is a homography the mathematically correct model to describe the geometric transformation between two images?
6.  What happens if you try to compute and apply a homography between two images of a non-planar (3D) scene where the camera has undergone significant translation (not just rotation)? Would you expect a perfect global alignment? Why or why not?

# Exercise 4: Measuring Lengths on a Plane using Homography

**Objective:** To use a homography transformation to undo perspective distortion in an image and perform metric measurements of an object (a line) on a known plane (a piece of paper).


**Instructions:**

1.  **Load the Image:**
    * Load the image (e.g., `line.jpg`) into your Python script using `cv2.imread()`.

2.  **Define Image Coordinates (Pixel Coordinates):**
    * You need to identify the pixel coordinates of specific points in your *distorted* image:
        * The **four corners of the sheet of paper**. List them in a consistent order (e.g., top-left, top-right, bottom-right, bottom-left).
        * The **two endpoints of the line** you want to measure.
    * **How to get these coordinates:**
        * **Option A (Manual Estimation):** Open the image in an image viewer that shows pixel coordinates as you hover your mouse (e.g., `plotly.express.imshow()` and then observe coordinates). Carefully note these down.
        * **Option B (Interactive Clicking - Advanced):** Write a small OpenCV script with a mouse callback function (`cv2.setMouseCallback`) to click on the points and record their coordinates.
    * Store these image coordinates as NumPy arrays. For the square, it will be a `4x2` array. For the line, a `2x2` array.

3.  **Define Real-World Coordinates of the Reference Square:**
    * Define the known real-world coordinates for the four corners of the A4 sheet of paper. These coordinates should correspond to the order you used for the image points.
    * For a 5x5cm square, if you want to measure in millimeters (mm), you could define the corners as:
        * Top-left: `(0, 0)`
        * Top-right: `(50, 0)` (assuming width is 50mm)
        * Bottom-right: `(50, 50)`
        * Bottom-left: `(0, 50)` (assuming height is 50mm)
    * Store these as a NumPy array, e.g., `world_pts_sheet`.

4.  **Compute the Homography Matrix (Image to World):**
    * Use `cv2.findHomography()` to find the perspective transformation matrix `H` that maps the *image coordinates* of the square's corners (`image_pts_sheet`) to their *real-world coordinates* (`world_pts_sheet`).
        * `H, status = cv2.findHomography(image_pts_sheet, world_pts_sheet)`
        * Note: We are mapping *from* the distorted image *to* the ideal, flat-plane world representation.

5.  **Transform Line Endpoints to Real-World Coordinates:**
    * You have the image coordinates of the line's endpoints (`image_pts_line`).
    * Use `cv2.perspectiveTransform(src, H)` to apply the computed homography `H` to these image points.
        * `src` must be a 3-channel array (or an array of 2-channel arrays). So, reshape your `image_pts_line` (e.g., `(N, 1, 2)` where N is number of points).
        * The result will be the coordinates of the line's endpoints in your defined real-world coordinate system (e.g., in millimeters on the plane of the paper).
    * Store these transformed points, e.g., `world_pts_line_transformed`.

6.  **Calculate the Length of the Line in Real-World Units:**
    * You now have the two endpoints of the line in real-world coordinates (e.g., `(x1_w, y1_w)` and `(x2_w, y2_w)`).
    * Calculate the Euclidean distance between these two points:
        * `length = sqrt((x2_w - x1_w)^2 + (y2_w - y1_w)^2)`
    * This `length` is your measurement of the line in the units you defined for your real-world square (e.g., millimeters).

7.  **Verification:**
    * Compare the actual length (150 mm) to the length calculated using the homography.

**Optional Visualization:**

* You can use the inverse of the homography `H_inv = np.linalg.inv(H)` (or compute a new homography from world to image) to warp the entire input image so that the sheet of paper appears perfectly rectangular ("bird's-eye view"). Then, you can draw your measured line on this rectified image.


In [ ]:
import math

image_path = "images/line.jpg"  # Replace with your image path
img = cv2.imread(image_path)

if img is None:
    print(f"Error: Could not load image at {image_path}")
    exit()

print(f"Image loaded successfully. Shape: {img.shape}")

# Example coordinates (these are placeholders - YOU MUST CHANGE THEM)
image_pts_sheet = np.array(
    [
        [150, 180],  # Top-Left corner of the square in the image
        [450, 150],  # Top-Right corner
        [480, 400],  # Bottom-Right corner
        [120, 430],  # Bottom-Left corner
    ],
    dtype="float32",
)

# Example coordinates for the line to be measured (YOU MUST CHANGE THEM)
image_pts_line = np.array(
    [
        [200, 250],  # Start point of the line in the image
        [400, 350],  # End point of the line in the image
    ],
    dtype="float32",
)

print("\nImage coordinates for reference square (pixels):")
print(image_pts_sheet)
print("\nImage coordinates for line endpoints (pixels):")
print(image_pts_line)

# --- 3. Define Real-World Coordinates of the Reference Square ---
# We want measurements in millimeters.
# Order must match the image_pts_sheet order.
world_pts_sheet = None  # Change this

print("\nReal-world coordinates for reference square (mm):")
print(world_pts_sheet)

# --- 4. Compute the Homography Matrix (Image to World) ---
# This matrix will transform points from the image plane to the real-world plane.
H, status = (None, None)  # Change this

if H is None:
    print("Error: Homography computation failed. Check your points.")
    exit()

print("\nComputed Homography Matrix (Image to World):")
print(H)

# --- 5. Transform Line Endpoints to Real-World Coordinates ---
# cv2.perspectiveTransform expects points in a specific shape: (N, 1, 2)
# where N is the number of points.
image_pts_line_reshaped = image_pts_line.reshape(-1, 1, 2)

world_pts_line_transformed = None  # Change this

if world_pts_line_transformed is None:
    print("Error: Perspective transform failed for line points.")
    exit()

# Reshape back for easier access if needed, e.g., N x 2
world_pts_line_transformed = world_pts_line_transformed.reshape(-1, 2)
print("\nTransformed real-world coordinates for line endpoints (mm):")
print(world_pts_line_transformed)

# --- 6. Calculate the Length of the Line in Real-World Units ---
length_in_mm = None  # Change this

print(f"\nCalculated length of the line: {length_in_mm:.2f} mm")

# Exercise 5: Robustness to Outliers - Homography Before & After RANSAC

**Objective:** To observe and understand the impact of outlier matches on homography computation and to demonstrate the effectiveness of RANSAC in mitigating these effects.

**Instructions:**

1.  **Load Images and Good Matches:**
    * If you haven't already, run your feature matching code (e.g., using ORB with Lowe's Ratio Test from Exercise 2) to obtain `image1_color`, `image2_color`, `kp1`, `kp2`, and the list `good_matches`.

2.  **Introduce Outlier (Bad) Matches:**
    * Create a new list of matches, say `matches_with_outliers`, by copying your `good_matches`.
    * **Deliberately add a few incorrect matches** to this new list. For example, take 3-5 keypoints from `image1_color` (using their indices from `kp1`) and pair them with completely unrelated keypoints from `image2_color` (using random or deliberately wrong indices from `kp2`).
    * To do this, you'll create new `cv2.DMatch` objects for these bad pairings and append them to `matches_with_outliers`.
        * A `cv2.DMatch` object typically takes `_queryIdx`, `_trainIdx`, and `_distance`. For these fake outliers, the distance can be set to a small arbitrary value if needed by some functions, though it won't be used by `findHomography`'s core logic when method 0 is used.
    * Print the total number of matches in `matches_with_outliers`.

3.  **Extract Coordinates for All Matches (including Outliers):**
    * From `matches_with_outliers`, extract the source points (`src_pts_all`) from `kp1` and destination points (`dst_pts_all`) from `kp2`.
    * Ensure these are NumPy arrays of shape `(N, 1, 2)` and type `float32`.

4.  **Compute Homography WITHOUT RANSAC:**
    * Use `cv2.findHomography()` with `method=0`. This method uses a simple least-squares approach on *all* provided points.
        * `H_no_ransac, _ = cv2.findHomography(src_pts_all, dst_pts_all, 0)`
    * Print the computed `H_no_ransac`.

5.  **Warp Image using Homography WITHOUT RANSAC:**
    * Use `cv2.warpPerspective()` to apply `H_no_ransac` to `image1_color`.
        * `image1_warped_no_ransac = cv2.warpPerspective(image1_color, H_no_ransac, (image2_color.shape[1], image2_color.shape[0]))`
    * Display `image2_color` (reference) and `image1_warped_no_ransac` side-by-side or blended. Observe the (likely poor) alignment.

6.  **Compute Homography WITH RANSAC:**
    * Now, use `cv2.findHomography()` with the `cv2.RANSAC` method and a `reprojThresh` (e.g., 5.0).
        * `H_ransac, mask_ransac = cv2.findHomography(src_pts_all, dst_pts_all, cv2.RANSAC, 5.0)`
    * Print the computed `H_ransac` and the number of inliers found (by summing `mask_ransac`).

7.  **Warp Image using Homography WITH RANSAC:**
    * Use `cv2.warpPerspective()` to apply `H_ransac` to `image1_color`.
        * `image1_warped_ransac = cv2.warpPerspective(image1_color, H_ransac, (image2_color.shape[1], image2_color.shape[0]))`
    * Display `image2_color` (reference) and `image1_warped_ransac` side-by-side or blended. Observe the (hopefully good) alignment.

8.  **Visualize Inlier/Outlier Mask:**
    * Use `cv2.drawMatches()` to visualize the matches from `matches_with_outliers`.
    * Use the `mask_ransac` (obtained from `findHomography` with RANSAC) as the `matchesMask` parameter in `cv2.drawMatches()`. This will draw lines only for inlier matches, or you can customize it to draw inliers and outliers in different colors.
    * `img_matches_with_mask = cv2.drawMatches(image1_color, kp1, image2_color, kp2, matches_with_outliers, None, matchesMask=mask_ransac.ravel().tolist(), flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)`
    * Display this image.

**Discussion Points:**

1.  Describe the visual result of `image1_warped_no_ransac`. How did the outliers affect the homography computation when RANSAC was not used?
2.  Describe the visual result of `image1_warped_ransac`. How did RANSAC improve the result?
3.  Examine the `mask_ransac`. How many of your deliberately added bad matches were correctly identified as outliers by RANSAC? Were any of the original "good" matches (if you can distinguish them) classified as outliers? Why might that happen?
4.  What is the fundamental principle behind RANSAC that allows it to be robust to outliers?
5.  If you increased the number of outliers significantly (e.g., 50% of matches are bad), how might that affect RANSAC's ability to find the correct homography? What parameter in `cv2.findHomography` relates to the number of iterations RANSAC performs, and why is it important?
6.  Besides `method=0` and `cv2.RANSAC`, what other methods are available for `cv2.findHomography` (e.g., `cv2.LMEDS`)? When might they be useful?

In [ ]:
def add_outliers_to_keypoints(matches):
    import random

    matches_with_outliers = list(good_matches)  # Start with a copy of good matches
    num_outliers_to_add = 5  # Let's add 5 bad matches

    if len(kp1) < num_outliers_to_add or len(kp2) < num_outliers_to_add:
        print(
            "Warning: Not enough keypoints to select distinct outliers. Reducing number of outliers."
        )
        num_outliers_to_add = min(len(kp1), len(kp2), num_outliers_to_add)

    # Ensure we have enough keypoints in kp1 and kp2 to pick from
    # And that we don't pick existing trainIdx for a queryIdx if possible (though not strictly enforced here for simplicity)
    for i in range(num_outliers_to_add):
        if not kp1 or not kp2:
            break  # Should not happen if previous checks passed

        # Pick a random keypoint from image 1 (query)
        idx_kp1 = random.randint(0, len(kp1) - 1)

        # Pick a completely random keypoint from image 2 (train) for the bad match
        # This chosen kp2 index should ideally not be the "correct" match for idx_kp1
        idx_kp2_bad = random.randint(0, len(kp2) - 1)

        # Create a new DMatch object for the bad match
        # Distance can be arbitrary for this demonstration, let's make it look "good" to fool least-squares
        bad_match = cv2.DMatch(
            _queryIdx=idx_kp1, _trainIdx=idx_kp2_bad, _distance=random.uniform(10, 30)
        )
        matches_with_outliers.append(bad_match)
        print(f"Added outlier: kp1[{idx_kp1}] matched with kp2[{idx_kp2_bad}]")
    return matches_with_outliers

In [ ]:
# --- (Assuming SIFT feature matching from Exercise 2 is done here) ---

# Use keypoints from above

# --- 2. Introduce Outlier (Bad) Matches ---
matches_with_outliers = add_outliers_to_keypoints(good_matches)

print(f"Total matches (good + outliers): {len(matches_with_outliers)}")

# --- 3. Extract Coordinates for All Matches (including Outliers) ---

src_pts_all = None  # Change this
dst_pts_all = None  # Change this

# --- 4. Compute Homography WITHOUT RANSAC ---
# Method 0 is the default least-squares method using all points
H_no_ransac, status_no_ransac = None

print("\nHomography Matrix WITHOUT RANSAC (method=0):")
if H_no_ransac is not None:
    print(H_no_ransac)
else:
    print("Failed to compute homography without RANSAC.")
    # We can still try RANSAC if this fails

# --- 5. Warp Image using Homography WITHOUT RANSAC ---
if H_no_ransac is not None:
    image1_warped_no_ransac = cv2.warpPerspective(
        image1_color, H_no_ransac, (image2_color.shape[1], image2_color.shape[0])
    )

    fig_no_ransac, axs_no_ransac = plt.subplots(1, 2, figsize=(12, 6))
    axs_no_ransac[0].imshow(cv2.cvtColor(image2_color, cv2.COLOR_BGR2RGB))
    axs_no_ransac[0].set_title("Reference Image 2")
    axs_no_ransac[0].axis("off")
    axs_no_ransac[1].imshow(cv2.cvtColor(image1_warped_no_ransac, cv2.COLOR_BGR2RGB))
    axs_no_ransac[1].set_title("Warped Image 1 (NO RANSAC)")
    axs_no_ransac[1].axis("off")
    plt.suptitle("Alignment WITHOUT RANSAC")
    plt.show()
else:
    print("Skipping warp without RANSAC as H_no_ransac is None.")


# --- 6. Compute Homography WITH RANSAC ---
reproj_thresh = 5.0  # Maximum reprojection error (pixels) for RANSAC
H_ransac, mask_ransac = None  # Change this

print("\nHomography Matrix WITH RANSAC:")
if H_ransac is not None:
    print(H_ransac)
    num_inliers = np.sum(mask_ransac)
    print(
        f"Number of inliers found by RANSAC: {num_inliers} out of {len(matches_with_outliers)}"
    )
else:
    print("Failed to compute homography WITH RANSAC. Check matches or reproj_thresh.")
    exit()  # Critical if this fails

# --- 7. Warp Image using Homography WITH RANSAC ---
image1_warped_ransac = cv2.warpPerspective(
    image1_color, H_ransac, (image2_color.shape[1], image2_color.shape[0])
)

fig_ransac, axs_ransac = plt.subplots(1, 2, figsize=(12, 6))
axs_ransac[0].imshow(cv2.cvtColor(image2_color, cv2.COLOR_BGR2RGB))
axs_ransac[0].set_title("Reference Image 2")
axs_ransac[0].axis("off")
axs_ransac[1].imshow(cv2.cvtColor(image1_warped_ransac, cv2.COLOR_BGR2RGB))
axs_ransac[1].set_title("Warped Image 1 (WITH RANSAC)")
axs_ransac[1].axis("off")
plt.suptitle("Alignment WITH RANSAC")
plt.show()

# --- 8. Visualize Inlier/Outlier Mask ---
# drawMatches will use the mask to draw only inliers if matchesMask is provided.
# To draw all matches but color them differently would require more custom drawing.
# Here, we'll just draw the inliers identified by RANSAC.
if mask_ransac is not None:
    # Create a list of DMatch objects that are inliers
    inlier_matches = [
        m for i, m in enumerate(matches_with_outliers) if mask_ransac[i][0] == 1
    ]

    img_inlier_matches = cv2.drawMatches(
        image1_color,
        kp1,
        image2_color,
        kp2,
        inlier_matches,
        None,  # Pass only inlier DMatch objects
        flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS,
    )

    # Alternative: draw all matches and use matchesMask parameter
    # img_matches_with_mask = cv2.drawMatches(image1_color, kp1, image2_color, kp2,
    #                                         matches_with_outliers, None,
    #                                         matchesMask=mask_ransac.ravel().tolist(), # Mask for all matches
    #                                         flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

    plt.figure(figsize=(16, 8))
    plt.imshow(cv2.cvtColor(img_inlier_matches, cv2.COLOR_BGR2RGB))
    plt.title(f"Inlier Matches Identified by RANSAC ({len(inlier_matches)} inliers)")
    plt.axis("off")
    plt.show()
else:
    print("mask_ransac is None, skipping inlier visualization.")